In [119]:
import pandas as pd
import gensim
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [120]:
# Load data
main_data = pd.read_csv('/Users/andrewsimon/Desktop/IMDBDataset.csv.zip')
main_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [121]:
# Map sentiments to 1's and zeros
main_data['sentiment'] = main_data['sentiment'].map({'positive':1, 'negative': 0})
main_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [122]:
# Preprocess data for W2v
main_data['review'] = main_data['review'].apply(lambda x: gensim.utils.simple_preprocess(x))
main_data.head()

,review,sentiment
0,"[one, of, the, other, reviewers, has, mentione...",1
1,"[wonderful, little, production, br, br, the, f...",1
2,"[thought, this, was, wonderful, way, to, spend...",1
3,"[basically, there, family, where, little, boy,...",0
4,"[petter, mattei, love, in, the, time, of, mone...",1


In [123]:
 main_data = main_data.sample(n=500, random_state=1516)
 main_data = main_data.reset_index()

In [124]:
# Load pretrained elmo model
elmo = hub.load('/Users/andrewsimon/Downloads/elmo_3').signatures['default']


In [126]:
for i in range(len(main_data['review'])):
    sub_list = []
    print(i)
    for j in main_data['review'][i]:
        try:
            embeddings_tensor = elmo(tf.constant([j]))['elmo']
            embeddings_tensor = embeddings_tensor.numpy()
            sub_list.append(embeddings_tensor)
        except KeyError:
            sub_list.append(np.zeros(300,))
    main_data['review'][i] = sub_list

0


/var/folders/8f/mjj28lvs5fb1m_c_tvmtckl80000gn/T/ipykernel_32300/3358581724.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['review'][i] = sub_list


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [99]:
for i in range(len(main_data['review'])):
    sub_list = []
    for j in main_data['review'][i]:
        try:
            embeddings_tensor = elmo(tf.constant([j]))['elmo']
            embeddings_tensor = embeddings_tensor.numpy()
            sub_list.append(embeddings_tensor)
        except KeyError:
            sub_list.append(np.zeros(300,))
    main_data['review'][i] = sub_list

/var/folders/8f/mjj28lvs5fb1m_c_tvmtckl80000gn/T/ipykernel_32300/2367334988.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['review'][i] = sub_list


In [133]:
# Split train, test data
X_train, X_test, y_train, y_test = train_test_split(main_data['review'], main_data['sentiment'], test_size=0.2, random_state=1516)

In [134]:
# Pass to numpy arrays
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [135]:
# Pad data to remove ragged arrays
training_padded = pad_sequences(X_train, maxlen=120, truncating='post')
testing_padded = pad_sequences(X_test, maxlen=120, truncating='post')

In [136]:
# Create model, evaluate

batch = 32

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', 'Recall', 'AUC', 'Precision', 'FalseNegatives', 'FalsePositives'])
    
num_epochs = 10
model.fit(training_padded, y_train, epochs=num_epochs,batch_size=batch, validation_data=(testing_padded, y_test))
model.summary()

Epoch 1/10
13/13 [==============================] - 1s 24ms/step - loss: 0.7003 - accuracy: 0.5400 - recall: 0.2418 - auc: 0.5113 - precision: 0.4889 - false_negatives: 138.0000 - false_positives: 46.0000 - val_loss: 0.6930 - val_accuracy: 0.5200 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_false_negatives: 48.0000 - val_false_positives: 0.0000e+00
Epoch 2/10
13/13 [==============================] - 0s 8ms/step - loss: 0.6572 - accuracy: 0.5825 - recall: 0.1703 - auc: 0.5715 - precision: 0.6596 - false_negatives: 151.0000 - false_positives: 16.0000 - val_loss: 0.6919 - val_accuracy: 0.5300 - val_recall: 0.0208 - val_auc: 0.5104 - val_precision: 1.0000 - val_false_negatives: 47.0000 - val_false_positives: 0.0000e+00
Epoch 3/10
13/13 [==============================] - 0s 9ms/step - loss: 0.6220 - accuracy: 0.6350 - recall: 0.3352 - auc: 0.6401 - precision: 0.7093 - false_negatives: 121.0000 - false_positives: 25.0000 - val_loss: 0.7234 - val_accuracy: 0.50

In [137]:
training_padded_elmo = training_padded
testing_padded_elmo = testing_padded

%store training_padded_elmo
%store testing_padded_elmo

Stored 'training_padded_elmo' (ndarray)
Stored 'testing_padded_elmo' (ndarray)


In [138]:
%store y_train
%store y_test

Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
